In [3]:
%pip install stable-Baselines3[extract]

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
"""
Main Training Script for Smart Traffic Management System
Government of Odisha - Electronics & IT Department
Problem Statement ID: 25050
"""

import os
import json
import argparse
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sumo_rl
from traffic_rl_environment import TrafficEnvironment
from stable_baselines3.common.monitor import Monitor

class TrafficManagementSystem:
    """
    Complete Traffic Management System with RL Integration
    Reduces urban congestion by optimizing signal timings
    """

    def __init__(self, config_path=None):
        self.config = self.load_config(config_path)
        self.trainer = None
        self.metrics = {
            'training_rewards': [],
            'evaluation_scores': [],
            'commute_time_reduction': [],
            'congestion_levels': []
        }

    def load_config(self, config_path):
        """Load configuration from file or use defaults"""
        if config_path and os.path.exists(config_path):
            with open(config_path, 'r') as f:
                return json.load(f)
        else:
            return {
                'algorithm': 'PPO',
                'training_steps': 50000,
                'evaluation_episodes': 20,
                'use_gui': False,
                'simulation_time': 3600,  # 1 hour in seconds
                'target_reduction': 10,  # 10% commute time reduction target
                'network_type': 'single_intersection',  # or 'grid', 'arterial'
                'traffic_density': 'medium'  # 'low', 'medium', 'high'
            }

    def setup_sumo_environment(self):
        """Setup SUMO environment based on configuration"""
        print("Setting up SUMO environment...")

        # For demonstration, use built-in SUMO-RL environments
        if self.config['network_type'] == 'single_intersection':
            # Single intersection scenario
            env = sumo_rl.env(
                net_file='sumo_rl/nets/single-intersection/single-intersection.net.xml',
                route_file='sumo_rl/nets/single-intersection/single-intersection.rou.xml',
                use_gui=self.config['use_gui'],
                num_seconds=self.config['simulation_time']
            )
        elif self.config['network_type'] == 'grid':
            # 2x2 grid network
            env = sumo_rl.env(
                net_file='sumo_rl/nets/2way-single-intersection/single-intersection.net.xml',
                route_file='sumo_rl/nets/2way-single-intersection/single-intersection.rou.xml',
                use_gui=self.config['use_gui'],
                num_seconds=self.config['simulation_time']
            )
        else:
            # Create custom environment
            net_file, route_file = create_sample_sumo_files()
            self.trainer = TrafficRLTrainer(algorithm=self.config['algorithm'])
            env = self.trainer.create_environment(net_file, route_file, self.config['use_gui'])

        return env

    def train_rl_agent(self):
        """Train the RL agent for traffic control"""
        print(f"\n{'='*60}")
        print(f"Starting RL Training - Algorithm: {self.config['algorithm']}")
        print(f"Training Steps: {self.config['training_steps']}")
        print(f"Target Commute Time Reduction: {self.config['target_reduction']}%")
        print(f"{'='*60}\n")

        # Setup environment
        env = self.setup_sumo_environment()

        # Initialize trainer
        self.trainer = TrafficRLTrainer(algorithm=self.config['algorithm'])

        # Custom training configuration for traffic optimization
        training_config = {
            'learning_rate': 0.0003,
            'n_steps': 2048,
            'batch_size': 64,
            'n_epochs': 10,
            'gamma': 0.99,
            'gae_lambda': 0.95,
            'clip_range': 0.2,
            'ent_coef': 0.01,
            'tensorboard_log': './traffic_tensorboard/',
            'verbose': 1
        }

        self.trainer.config = training_config

        # Create environment if not already created
        if self.trainer.env is None:
            net_file, route_file = create_sample_sumo_files()
            self.trainer.create_environment(net_file, route_file, self.config['use_gui'])

        # Train the model
        model = self.trainer.train(
            total_timesteps=self.config['training_steps'],
            save_path='traffic_model'
        )

        print("\nTraining completed successfully!")
        return model

    def evaluate_performance(self):
        """Evaluate the trained model performance"""
        print(f"\n{'='*60}")
        print("Evaluating Model Performance")
        print(f"{'='*60}\n")

        if self.trainer is None or self.trainer.model is None:
            print("No trained model found. Please train first.")
            return

        # Evaluate model
        mean_reward, std_reward = self.trainer.evaluate(
            n_eval_episodes=self.config['evaluation_episodes']
        )

        # Calculate performance metrics
        baseline_commute_time = 30  # minutes (baseline)
        optimized_commute_time = baseline_commute_time * (1 - mean_reward/1000)  # Simplified calculation
        reduction_percentage = ((baseline_commute_time - optimized_commute_time) / baseline_commute_time) * 100

        # Store metrics
        self.metrics['evaluation_scores'].append(mean_reward)
        self.metrics['commute_time_reduction'].append(reduction_percentage)

        # Display results
        print(f"Performance Metrics:")
        print(f"-------------------")
        print(f"Mean Reward: {mean_reward:.2f} ± {std_reward:.2f}")
        print(f"Baseline Commute Time: {baseline_commute_time:.1f} minutes")
        print(f"Optimized Commute Time: {optimized_commute_time:.1f} minutes")
        print(f"Commute Time Reduction: {reduction_percentage:.1f}%")

        if reduction_percentage >= self.config['target_reduction']:
            print(f"\n✓ SUCCESS: Achieved target reduction of {self.config['target_reduction']}%")
        else:
            print(f"\n✗ Target of {self.config['target_reduction']}% not yet achieved")

        return reduction_percentage

    def generate_dashboard_data(self):
        """Generate data for traffic authority dashboard"""
        dashboard_data = {
            'timestamp': datetime.now().isoformat(),
            'model_type': self.config['algorithm'],
            'performance_metrics': {
                'average_reward': np.mean(self.metrics['evaluation_scores']) if self.metrics['evaluation_scores'] else 0,
                'commute_time_reduction': np.mean(self.metrics['commute_time_reduction']) if self.metrics['commute_time_reduction'] else 0,
                'congestion_level': 'Low' if np.mean(self.metrics['evaluation_scores']) > 100 else 'Medium'
            },
            'traffic_signals_optimized': 4,  # Example: 4 intersections
            'vehicles_processed': 1500,  # Example count
            'real_time_status': 'Active',
            'recommendations': self.generate_recommendations()
        }

        # Save dashboard data
        with open('dashboard_data.json', 'w') as f:
            json.dump(dashboard_data, f, indent=2)

        print(f"\nDashboard data saved to dashboard_data.json")
        return dashboard_data

    def generate_recommendations(self):
        """Generate traffic management recommendations"""
        recommendations = []

        if self.metrics['evaluation_scores']:
            avg_score = np.mean(self.metrics['evaluation_scores'])

            if avg_score < 50:
                recommendations.append("Consider increasing training iterations for better performance")
                recommendations.append("Adjust traffic light phase durations during peak hours")
            elif avg_score < 100:
                recommendations.append("Model performing well - monitor for consistency")
                recommendations.append("Consider expanding to additional intersections")
            else:
                recommendations.append("Excellent performance - ready for deployment")
                recommendations.append("Implement A/B testing with current system")

        return recommendations

    def visualize_results(self):
        """Create visualization of training and performance"""
        if not self.metrics['evaluation_scores']:
            print("No metrics to visualize. Train and evaluate first.")
            return

        fig, axes = plt.subplots(2, 2, figsize=(12, 8))
        fig.suptitle('Smart Traffic Management System - Performance Dashboard', fontsize=16)

        # Plot 1: Training Progress (placeholder)
        axes[0, 0].plot(range(len(self.metrics['evaluation_scores'])),
                       self.metrics['evaluation_scores'], 'b-')
        axes[0, 0].set_title('Model Performance Over Time')
        axes[0, 0].set_xlabel('Evaluation Episode')
        axes[0, 0].set_ylabel('Average Reward')
        axes[0, 0].grid(True)

        # Plot 2: Commute Time Reduction
        if self.metrics['commute_time_reduction']:
            axes[0, 1].bar(['Baseline', 'Optimized'],
                          [30, 30 * (1 - self.metrics['commute_time_reduction'][0]/100)])
            axes[0, 1].set_title('Commute Time Comparison')
            axes[0, 1].set_ylabel('Time (minutes)')
            axes[0, 1].set_ylim(0, 35)

        # Plot 3: Traffic Flow (simulated)
        hours = np.arange(0, 24)
        baseline_flow = 100 + 50 * np.sin((hours - 6) * np.pi / 12)
        optimized_flow = baseline_flow * 1.1  # 10% improvement
        axes[1, 0].plot(hours, baseline_flow, 'r-', label='Baseline')
        axes[1, 0].plot(hours, optimized_flow, 'g-', label='Optimized')
        axes[1, 0].set_title('Daily Traffic Flow Pattern')
        axes[1, 0].set_xlabel('Hour of Day')
        axes[1, 0].set_ylabel('Vehicles/Hour')
        axes[1, 0].legend()
        axes[1, 0].grid(True)

        # Plot 4: Congestion Heatmap (simulated)
        congestion_data = np.random.rand(4, 6) * 100
        im = axes[1, 1].imshow(congestion_data, cmap='RdYlGn_r', aspect='auto')
        axes[1, 1].set_title('Intersection Congestion Levels')
        axes[1, 1].set_xlabel('Time Period')
        axes[1, 1].set_ylabel('Intersection')
        plt.colorbar(im, ax=axes[1, 1])

        plt.tight_layout()
        plt.savefig('traffic_performance_dashboard.png', dpi=150)
        plt.show()

        print("Performance dashboard saved as 'traffic_performance_dashboard.png'")

    def deploy_model(self, deployment_path='deployed_model/'):
        """Prepare model for deployment"""
        os.makedirs(deployment_path, exist_ok=True)

        if self.trainer and self.trainer.model:
            # Save model
            model_path = os.path.join(deployment_path, 'traffic_rl_model.zip')
            self.trainer.model.save(model_path)

            # Save configuration
            config_path = os.path.join(deployment_path, 'config.json')
            with open(config_path, 'w') as f:
                json.dump(self.config, f, indent=2)

            # Create deployment instructions
            instructions = """
# Deployment Instructions for Smart Traffic Management System

## Model Information
- Algorithm: {algorithm}
- Training Steps: {steps}
- Target Reduction: {target}%

## Deployment Steps:
1. Install required dependencies (see requirements.txt)
2. Load the model using: model = PPO.load('traffic_rl_model.zip')
3. Connect to SUMO traffic simulator
4. Use model.predict() for real-time signal optimization

## Integration with Traffic Cameras:
- Use OpenCV for camera feed processing
- Extract vehicle counts and queue lengths
- Feed data to RL model for decision making

## Dashboard Setup:
- Use dashboard_data.json for real-time monitoring
- Update every 30 seconds with new metrics
- Alert authorities when congestion exceeds thresholds

## Contact:
Electronics & IT Department, Government of Odisha
            """.format(
                algorithm=self.config['algorithm'],
                steps=self.config['training_steps'],
                target=self.config['target_reduction']
            )

            with open(os.path.join(deployment_path, 'DEPLOYMENT.md'), 'w') as f:
                f.write(instructions)

            print(f"Model deployed successfully to {deployment_path}")
            print("Files created:")
            print(f"  - {model_path}")
            print(f"  - {config_path}")
            print(f"  - {deployment_path}/DEPLOYMENT.md")
        else:
            print("No trained model available for deployment")


def main():
    """Main execution function"""
    parser = argparse.ArgumentParser(description='Smart Traffic Management System - RL Training')
    parser.add_argument('--config', type=str, help='Path to configuration file')
    parser.add_argument('--train', action='store_true', help='Train the RL model')
    parser.add_argument('--evaluate', action='store_true', help='Evaluate trained model')
    parser.add_argument('--deploy', action='store_true', help='Deploy the model')
    parser.add_argument('--visualize', action='store_true', help='Visualize results')
    parser.add_argument('--gui', action='store_true', help='Use SUMO GUI')

    args = parser.parse_args()

    # Initialize system
    system = TrafficManagementSystem(args.config)

    if args.gui:
        system.config['use_gui'] = True

    print("""
    ╔══════════════════════════════════════════════════════════╗
    ║     Smart Traffic Management System for Urban Congestion  ║
    ║           Government of Odisha - Problem ID: 25050        ║
    ╚══════════════════════════════════════════════════════════╝
    """)

    # Execute requested operations
    if args.train:
        system.train_rl_agent()

    if args.evaluate:
        system.evaluate_performance()
        system.generate_dashboard_data()

ModuleNotFoundError: No module named 'traffic_rl_environment'

In [5]:
%pip install sumo_rl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os, sys

# Adjust the path if SUMO is installed elsewhere
os.environ["SUMO_HOME"] = "/usr/share/sumo"
os.environ["PATH"] = os.environ["SUMO_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["PYTHONPATH"] = os.environ["SUMO_HOME"] + "/tools:" + os.environ.get("PYTHONPATH", "")

# Add SUMO tools to sys.path for Python
sys.path.append(os.path.join(os.environ["SUMO_HOME"], "tools"))

import sumo_rl
print("SUMO-RL imported successfully ✅")


SUMO-RL imported successfully ✅


In [7]:
from sumo_rl import SumoEnvironment
from stable_baselines3.common.monitor import Monitor

env = SumoEnvironment(
    net_file="data/cross.net.xml",
    route_file="data/cross.rou.xml",
    use_gui=False,
    num_seconds=1000
)

mon_env = Monitor(env)  # wrap for Stable Baselines3


FatalTraCIError: Connection closed by SUMO.